In [1]:
import requests
import pandas as pd
import time


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Cell used to get all the different freebase etnictities from our characters.

In [3]:
#open the character.metadata.tsv file
characters = pd.read_csv('movie_summaries_ada/character.metadata.tsv', sep='\t', header=None)
#define the columns of the character file
characters.columns = ['wikipedia_movie_id','freebase_movie_id','movie_release_date','character_name','actor_birth',
                      'actor_gender','actor_height','actor_etnicity','actor_name','actor_age_at_release','freebase_char_actor_map_id','freebase_character_id','freebase_actor_id']

# list of all unique actor_ethnicity in the character file
etnicities = characters.actor_etnicity.unique()

In [6]:
def get_link(freebase_id):
    url = "https://query.wikidata.org/sparql"
    query = """
    SELECT ?item WHERE {
      ?item wdt:P646 '""" + freebase_id + """'.
    }
    """
    response = requests.get(url, params={'format': 'json', 'query': query})
    data = response.json()
    results = data['results']['bindings']
    if results:
        wikidata_entity = results[0]['item']['value'].split('/')[-1]  # Extract Q-code
        return wikidata_entity
    else:
        return None

           
def get_values(wikidata_entity):
    url = "https://www.wikidata.org/wiki/Special:EntityData/" + wikidata_entity + ".json"
    response = requests.get(url)
    data = response.json()

    # Here you can parse the JSON to extract whatever information you need
    # This is a basic example to get the English label and description
    entity_data = data['entities'][wikidata_entity]
    label = entity_data['labels']['en']['value'] if 'en' in entity_data['labels'] else "No label"
    description = entity_data['descriptions']['en']['value'] if 'en' in entity_data['descriptions'] else "No description"

    return label, description

def get_label(freebase_id):
    wikidata_entity = get_link(freebase_id)
    if wikidata_entity:
        label, description = get_values(wikidata_entity)
        print(freebase_id, "->", label)
        return label
    else:
        return None

# Example usage
freebase_id = '/m/0x67'  # Replace with your Freebase ID
wikidata_entity = get_link(freebase_id)

if wikidata_entity:
    label, description = get_values(wikidata_entity)
    print("Label:", label)
    print("Description:", description)
else:
    print("No corresponding Wikidata entity found.")

# Example usage with just the label
label = get_label(freebase_id)
print("Only Label:",label)

Label: African Americans
Description: Americans with sub-Saharan African ancestry
/m/0x67 -> African Americans
Only Label: African Americans


In [4]:
etnicities_dict = {}

In [ ]:
for freebase_id in etnicities:
    wikidata_entity = get_link(freebase_id)
    if wikidata_entity:
        label, description = get_values(wikidata_entity)
        etnicities_dict[freebase_id] = label
        print("Label:", label)
    else:
        print("No corresponding Wikidata entity found.")
    time.sleep(1) # sleep for 1 second to avoid hitting the API too quickly